<a href="https://colab.research.google.com/github/Tiger-A/Yandex_Data-Science/blob/master/Copy_of_%D0%A1%D0%B2%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0_%D0%B2_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Сводная%20таблица%20в%20pandas.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

*Сводная таблица* - это мощный инструмент для обобщения и представления данных. 

В Pandas есть функция [`DataFrame.pivot_table()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot_table.html), которая позволяет быстро преобразовать [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) в сводную таблицу.

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

Обобщенная схема работы функции `pivot_table`:

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pivot_table_pandas.png" >

Эта функция очень полезна, но иногда бывает сложно запомнить, как ее использовать для форматирования данных нужным вам способом.

В этом Блокноте рассказывается, как использовать `pivot_table`.

Полный текст оригинальной статьи находится [здесь](http://pbpython.com/pandas-pivot-table-explained.html).

В этом сценарии я собираюсь отслеживать воронку (план) продаж (также называемую воронкой, funnel). Основная проблема заключается в том, что некоторые циклы продаж очень длинные (например, "корпоративное программное обеспечение", капитальное оборудование и т.д.), и руководство хочет отслеживать их детально в течение года.

Типичные вопросы, относящиеся к таким данным, включают: 
- Какой доход находится в воронке (плане продаж)? 
- Какие продукты находятся в воронке? 
- У кого какие продукты на каком этапе? 
- Насколько вероятно, что мы закроем сделки к концу года?

In [ ]:
import pandas as pd
import numpy as np

Прочтите данные о нашей воронке продаж в `DataFrame`:

In [ ]:
df = pd.read_excel("https://github.com/dm-fedorov/pandas_basic/raw/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/salesfunnel.xlsx")
df.head()
# Счет, Название компании, Представитель компании, Менеджер по продажам, Продукт, Кол-во, Стоимость, Статус сделки

Для удобства давайте представим столбец `Status` как [категориальную переменную](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) (`category`) и установим порядок, в котором хотим просматривать.

Это не является строго обязательным, но помогает поддерживать желаемый порядок при работе с данными.

In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won", "pending", "presented", "declined"], inplace=True)

In [ ]:
df.info()

# Поворот данных

Создавать сводную таблицу (`pivot table`) проще всего последовательно. Добавляйте элементы по одному и проверяйте каждый шаг, чтобы убедиться, что вы получаете ожидаемые результаты.

Самая простая сводная таблица должна иметь `DataFrame` и индекс (`index`). В этом примере давайте использовать `Name` в качестве индекса:

In [ ]:
pd.pivot_table(df, 
               index=["Name"])

У вас может быть несколько индексов. Фактически, большинство аргументов `pivot_table` могут принимать несколько значений в качестве элементов списка:

In [ ]:
pd.pivot_table(df, 
               index=["Name", "Rep", "Manager"])

Это интересно, но не особо полезно. 

Мы хотим посмотреть на эти данные со стороны менеджера (`Manager`) и директора (`Director`). Это достаточно просто сделать, изменив индекс:

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"])

Вы могли заметить, что сводная таблица достаточно умна, чтобы начать агрегирование данных и их обобщение, группируя представителей (`Rep`) с их менеджерами (`Manager`). Теперь мы начинаем понимать, что может сделать для нас сводная таблица.

Давайте удалим счет (`Account`) и количество (`Quantity`), явно определив столбцы, которые нам нужны, с помощью параметра `values`:

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               values=["Price"])

Столбец цен (`price`) по умолчанию усредняет данные, но мы можем произвести подсчет количества или суммы. Добавить их можно с помощью параметра `aggfunc`:

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               values=["Price"], 
               aggfunc=np.sum)

`aggfunc` может принимать список функций. 

Давайте попробуем узнать среднее значение и количество:

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               values=["Price"], 
               aggfunc=[np.mean, len])

Если мы хотим увидеть продажи с разбивкой по продуктам (`Product`), переменная `columns` позволяет нам определить один или несколько столбцов.

Я думаю, что одна из сложностей `pivot_table` - это использование столбцов (`columns`) и значений (`values`). 

Помните, что столбцы необязательны - они предоставляют дополнительный способ сегментировать актуальные значения, которые вам нужны. 

Функции агрегирования применяются к перечисленным значениям (`values`):

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               values=["Price"],
               columns=["Product"], 
               aggfunc=[np.sum])

Значения `NaN` немного отвлекают. Если мы хотим их убрать, то можем использовать параметр `fill_value`, чтобы установить в `0`.

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Rep"],
               values=["Price"],
               columns=["Product"],
               aggfunc=[np.sum],
               fill_value=0)

Думаю, было бы полезно добавить количество (`Quantity`). 

Добавьте количество (`Quantity`) в список значений `values`:

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Rep"],
               values=["Price", "Quantity"],
               columns=["Product"],
               aggfunc=[np.sum],
               fill_value=0)

Что интересно, вы можете добавлять элементы в индекс, чтобы получить другое визуальное представление. 

Добавим товары (`Products`) в индекс.

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Rep", "Product"],
               values=["Price", "Quantity"],
               aggfunc=[np.sum],
               fill_value=0)

Для этого набора данных такое представление имеет больше смысла. 

А что, если я хочу увидеть некоторые итоги? `margins=True` делает это за нас.

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Rep", "Product"],
               values=["Price", "Quantity"],
               aggfunc=[np.sum, np.mean],
               fill_value=0,
               margins=True)

Давайте переместим анализ на уровень выше и посмотрим на наш план продаж (воронку) на уровне менеджера. 

Обратите внимание на то, как статус упорядочен на основе нашего предыдущего определения категории.

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Status"],
               values=["Price"],
               aggfunc=[np.sum],
               fill_value=0,
               margins=True)

Очень удобно передать словарь в качестве `aggfunc`, чтобы вы могли выполнять разные функции с каждым из выбранных значений. Это имеет побочный эффект - названия становятся немного чище:

In [ ]:
pd.pivot_table(df,
               index=["Manager", "Status"],
               columns=["Product"],
               values=["Quantity", "Price"],
               aggfunc={"Quantity": len, "Price": np.sum},
               fill_value=0)

Вы также можете предоставить список агрегированных функций (aggfunctions), которые будут применяться к каждому значению:

In [ ]:
table = pd.pivot_table(df,
                       index=["Manager", "Status"],
                       columns=["Product"],
                       values=["Quantity", "Price"],
                       aggfunc={"Quantity": len, "Price": [np.sum, np.mean]},
                       fill_value=0)

In [ ]:
table

Может показаться сложным собрать все это сразу, но как только вы начнете играть с данными и медленно добавлять элементы, то почувствуете, как это работает.

Мое общее практическое правило заключается в том, что после использования нескольких группировок (`grouby`) вы должны оценить, является ли сводная таблица (`pivot table`) полезным подходом.

# Расширенная фильтрация сводной таблицы

После того, как вы сгенерировали свои данные, они находятся в `DataFrame`, поэтому можно фильтровать их, используя обычные методы `DataFrame`.

Если вы хотите посмотреть только на одного менеджера:

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html

table.query('Manager == ["Debra Henley"]')

Мы можем просмотреть все незавершенные (`pending`) и выигранные (`won`) сделки:

In [ ]:
table.query('Status == ["pending", "won"]')

Я надеюсь, что этот пример показал вам, как использовать сводные таблицы в собственных наборах данных.

# Шпаргалка

Схема с примером из Блокнота: 

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pivot-table-datasheet.png" >

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>